### CA polygon

In [ ]:
%%sql
-- create original polygon table
CREATE TABLE ca_polygon (column1 TEXT, ...);

-- load the data to mysql table
LOAD DATA INFILE './CA_polygon.csv'
INTO TABLE ca_polygon
FIELDS TERMINATED BY ','   
ENCLOSED BY '"'             
LINES TERMINATED BY '\r\n'    
IGNORE 1 ROWS;     

-- create udf function: 
CREATE FUNCTION <function_name> RETURNS <STRING/INTEGER/REAL> SONAME '<udf_file_name>.so';


-- Rapid process

-- Features calculation
CREATE TABLE ca_polygon_par AS 
SELECT 
    polygon_area(coordinate) AS area, 
    polygon_perimeter(coordinate) AS perimeter, 
    polygon_center(coordinate, "lon") AS lon, 
    polygon_center(coordinate, "lat") AS lat 
FROM ca_polygon;

-- PIG calculation
CREATE TABLE ca_polygon_index_res01 AS 
SELECT 
    area, 
    perimeter, 
    grid_index_cal(lon, lat, CAST(-180 AS DOUBLE), CAST(180 AS DOUBLE), CAST(-90 AS DOUBLE), CAST(90 AS DOUBLE), CAST(0.1 AS DOUBLE)) AS grid_id 
FROM ca_polygon_par;

-- Aggregation
CREATE TABLE ca_result_res01 AS 
SELECT 
    grid_id, 
    COUNT(*) AS count, 
    SUM(area) AS area, 
    SUM(perimeter) AS perimeter, 
    grid_geometry_gen(grid_id, CAST(-180 AS DOUBLE), CAST(180 AS DOUBLE), CAST(-90 AS DOUBLE), CAST(90 AS DOUBLE), CAST(0.1 AS DOUBLE)) AS geometry 
FROM ca_polygon_index_res01
GROUP BY grid_id;